In [49]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [50]:
url_inicial = 'https://www.revistahistoriaygrafia.com.mx/index.php/HyG/issue/archive/5'
url_root = 'https://www.revistahistoriaygrafia.com.mx/index.php/HyG/issue/archive/5'
r=requests.get(url_inicial)

In [51]:
soup = BeautifulSoup(r.text, 'html.parser')

In [52]:
box = soup.find('ul', class_='issues_archive')
volumen=box.findAll('div', class_='obj_issue_summary')
vol = [x.find('a').get('href')for x in volumen]
vol=[urljoin(url_root,i) for i in vol]
vol

['https://www.revistahistoriaygrafia.com.mx/index.php/HyG/issue/view/7',
 'https://www.revistahistoriaygrafia.com.mx/index.php/HyG/issue/view/5',
 'https://www.revistahistoriaygrafia.com.mx/index.php/HyG/issue/view/4',
 'https://www.revistahistoriaygrafia.com.mx/index.php/HyG/issue/view/28',
 'https://www.revistahistoriaygrafia.com.mx/index.php/HyG/issue/view/23']

In [53]:
def get_url_items(sopa, url): 
    box = soup.find('ul', class_='issues_archive')
    volumen=box.findAll('div', class_='obj_issue_summary')
    vol = [x.find('a').get('href')for x in volumen]
    vol=[urljoin(url_root,i) for i in vol]
    return vol

In [54]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links)

Estoy en la pagina https://www.revistahistoriaygrafia.com.mx/index.php/HyG/issue/archive/5


In [55]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

5

In [56]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [57]:
#funcion para iniciar el escraper de cada libro
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tire='Historia y Grafía'
    a='IV. Humanidades y Ciencias de la Conducta'
    tem='Historia'
    url='http://www.revistahistoriaygrafia.com.mx'
    s_item=BeautifulSoup(r.text,'html.parser')
    #titulo de revista
    try:
        titulo=tire
        content_book['Titulo de revista']=tire
    except AttributeError:
        content_book['Titulo de revista']=None
    #area
    try:
        area=a
        content_book['Área']=a
    except AttributeError:
        content_book['Área']=None
    #tematica
    try:
        tema=tem
        content_book['Temática']=tem
    except AttributeError:
        content_book['Temática']=None
    #titulo libro
    try:
        titu=s_item.find('h1').get_text(strip=True)
        content_book['Titulo del libro']=titu
    except AttributeError:
        content_book['Titulo del libro']=None
    #resumen
    try:
        resu=s_item.find('div', class_='description').get_text(strip=True)
        content_book['Resumen']=resu
    except AttributeError:
        content_book['Resumen']=None
    #Link incial
    try:
        inicial=url
        content_book['Link inicial']=url
    except AttributeError:
        content_book['Link inicial']=None
    #link articulo
    try:
        link=s_item.find('a', class_='obj_galley_link pdf').get('href')
        content_book['Link articulo']=urljoin(url_root, link)
    except AttributeError:
        content_book['Link articulo']=None
    return content_book

In [58]:
list_scraper=list_scraper[0:5]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

estas escrapeando la pag 0
estas escrapeando la pag 1
estas escrapeando la pag 2
estas escrapeando la pag 3
estas escrapeando la pag 4


In [59]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

,Titulo de revista,Área,Temática,Titulo del libro,Resumen,Link inicial,Link articulo
0,Historia y Grafía,IV. Humanidades y Ciencias de la Conducta,Historia,Núm. 39 (2012): LA PRODUCCIÓN CINEMÁTICA DEL P...,None,http://www.revistahistoriaygrafia.com.mx,https://www.revistahistoriaygrafia.com.mx/inde...
1,Historia y Grafía,IV. Humanidades y Ciencias de la Conducta,Historia,Núm. 38 (2012): DECONSTRUYENDO EL ARCHIVO,None,http://www.revistahistoriaygrafia.com.mx,https://www.revistahistoriaygrafia.com.mx/inde...
2,Historia y Grafía,IV. Humanidades y Ciencias de la Conducta,Historia,Núm. 37 (2011): SIN CONTRINCANTE EN EL FRENTE....,None,http://www.revistahistoriaygrafia.com.mx,https://www.revistahistoriaygrafia.com.mx/inde...
3,Historia y Grafía,IV. Humanidades y Ciencias de la Conducta,Historia,Núm. 36 (2011): Siegfried Kracauer: las ambigü...,"Dedicado a Siegrfried Kracauer, el expediente ...",http://www.revistahistoriaygrafia.com.mx,https://www.revistahistoriaygrafia.com.mx/inde...
4,Historia y Grafía,IV. Humanidades y Ciencias de la Conducta,Historia,Núm. 1 (1993): Michel de Certeau: historia y a...,None,http://www.revistahistoriaygrafia.com.mx,https://www.revistahistoriaygrafia.com.mx/inde...


In [60]:
df_catalogo.to_csv('Revista209.05.csv', index=False)